In [1]:
!pip install researchpy

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import gensim
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import csv
from csv import reader
from scipy import spatial
import functools
from collections import Counter
import nltk
from nltk.data import load
tagdict = load('help/tagsets/upenn_tagset.pickle')

import imblearn
from imblearn.under_sampling import RandomUnderSampler

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate

import researchpy as rp
import scipy.stats as stats

from sklearn.metrics import confusion_matrix
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [3]:
%%time
# open file in read mode
with open('/kaggle/input/utf8tokenizedspeeches/TokenizedSpeeches_utf-8.csv', 'r') as read_obj:
    lemmasList = []
    
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    for row in csv_reader: 
        lemmasList.append(row)
    print(len(lemmasList), 'Rows read')
    

#create dataframe from the lemmas extracted from csv
dictOfLemmas = {'Lemmas': lemmasList}
lemmasDf = pd.DataFrame(dictOfLemmas)

with open('/kaggle/input/hansard-speeches-lemmatized/hansard-speeches-post2010.pkl', 'rb') as f:
    df = pickle.load(f)
    
#since index was missing values and didn't match with the lemmasDf index
df = df.reset_index(drop=True)
df = df.join(lemmasDf)

In [4]:
# Split data based on the Brexit referendum event before and after period
eventDate = '2016-06-23 23:59:59'
df_t1 = df[df['date']<= eventDate]
df_t2 = df[df['date']> eventDate]

In [5]:
%%time
model1 = gensim.models.Word2Vec.load('../input/models12/model1')
model2 = gensim.models.Word2Vec.load('../input/models12/model2')

In [6]:
def intersection_align_gensim(m1, m2, words=None):
    """
    Intersect two gensim word2vec models, m1 and m2.
    Only the shared vocabulary between them is kept.
    If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
    Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
    These indices correspond to the new syn0 and syn0norm objects in both gensim models:
        -- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
        -- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
    The .vocab dictionary is also updated for each model, preserving the count but updating the index.
    """
    # Get the vocab for each model
    vocab_m1 = set(m1.wv.index_to_key)
    vocab_m2 = set(m2.wv.index_to_key)

    # Find the common vocabulary
    common_vocab = vocab_m1 & vocab_m2
    if words: common_vocab &= set(words)
        
    # If no alignment necessary because vocab is identical...
    if not vocab_m1 - common_vocab and not vocab_m2 - common_vocab:
        return (m1,m2)

    # Otherwise sort by frequency (summed for both)
    common_vocab = list(common_vocab)
    common_vocab.sort(key=lambda w: m1.wv.get_vecattr(w, "count") + m2.wv.get_vecattr(w, "count"), reverse=True)
    # Then for each model...
    for m in [m1, m2]:
        # Replace old syn0norm array with new one (with common vocab)
        indices = [m.wv.key_to_index[w] for w in common_vocab]
        old_arr = m.wv.vectors
        new_arr = np.array([old_arr[index] for index in indices])
        m.wv.vectors = new_arr

        # Replace old vocab dictionary with new one (with common vocab)
        # and old index2word with new one
        new_key_to_index = {}
        new_index_to_key = []
        for new_index, key in enumerate(common_vocab):
            new_key_to_index[key] = new_index
            new_index_to_key.append(key)
        m.wv.key_to_index = new_key_to_index
        m.wv.index_to_key = new_index_to_key
        
    print('Vocab function returning models with shapes', m1.wv.vectors.shape, m2.wv.vectors.shape)    
    return (m1,m2)

In [7]:
# Function to align two spaces with orthogunal procrustes
def smart_procrustes_align_gensim(base_embed, other_embed, words=None):
    print('shapes', base_embed.wv.vectors.shape, other_embed.wv.vectors.shape)
    """
    Original script: https://gist.github.com/quadrismegistus/09a93e219a6ffc4f216fb85235535faf
    Procrustes align two gensim word2vec models (to allow for comparison between same word across models).
    Code ported from HistWords <https://github.com/williamleif/histwords> by William Hamilton <wleif@stanford.edu>.
        
    First, intersect the vocabularies (see `intersection_align_gensim` documentation).
    Then do the alignment on the other_embed model.
    Replace the other_embed model's syn0 and syn0norm numpy matrices with the aligned version.
    Return other_embed.
    If `words` is set, intersect the two models' vocabulary with the vocabulary in words (see `intersection_align_gensim` documentation).
    """

    # make sure vocabulary and indices are aligned
    in_base_embed, in_other_embed = intersection_align_gensim(base_embed, other_embed, words=words)

    # re-filling the normed vectors
    in_base_embed.wv.fill_norms(force=True)
    in_other_embed.wv.fill_norms(force=True)
    
    # get the (normalized) embedding matrices
    base_vecs = in_base_embed.wv.get_normed_vectors()
    
    other_vecs = in_other_embed.wv.get_normed_vectors()
    # just a matrix dot product with numpy
    m = other_vecs.T.dot(base_vecs) 

    # SVD method from numpy
    u, _, v = np.linalg.svd(m)

    # another matrix operation
    ortho = u.dot(v) 

    # Replace original array with modified one, i.e. multiplying the embedding matrix by "ortho"
    other_embed.wv.vectors = (other_embed.wv.vectors).dot(ortho)    
    print('Procrustes function returning')
    return other_embed

In [8]:
%%time
# Applying the functions to our models

smart_procrustes_align_gensim(model1, model2, words=None)

In [9]:
def cosine_similarity(word):
  sc = 1-spatial.distance.cosine(model1.wv[word], model2.wv[word])
  return sc

In [10]:
%%time
cosine_similarity_df = pd.DataFrame(([w, cosine_similarity(w), model1.wv.get_vecattr(w, "count") , model2.wv.get_vecattr(w, "count") ] for w in model1.wv.index_to_key), columns = ('Word', 'Cosine_similarity', "Frequency_t1", "Frequency_t2"))

cosine_similarity_df['FrequencyRatio'] = cosine_similarity_df['Frequency_t1']/cosine_similarity_df['Frequency_t2']
cosine_similarity_df['TotalFrequency'] = cosine_similarity_df['Frequency_t1'] + cosine_similarity_df['Frequency_t2']

cosine_similarity_df_sorted = cosine_similarity_df.sort_values(by='Cosine_similarity', ascending=True)

In [11]:
change = ['exiting', 'seaborne', 'eurotunnel', 'withdrawal', 'departures', 'unicorn', 'remainers', 'exit', 'surrender',
          'departure', 'triggering', 'stockpiling', 'expulsion', 'blindfold', 'cliff', 'lighter', 'exits', 'triggered',
          'brexiteer', 'soft', 'plus', 'trigger', 'backroom', 'invoked', 'protesting', 'brexit', 'edge', 'canary', 
          'unicorns', 'withdrawing', 'invoking', 'withdrawn', 'manor', 'brexiteers', 'fanatics', 'postponement', 
          'currencies', 'currency', 'operability', 'operable', 'leavers', 'invoke', 'article', 'eurozone', 'clueless',
          'surrendered', 'cake', 'red', 'euroscepticism', 'prorogation', 'lining', 'gove', 'norway', 'deflationary',
          'moribund', 'eurosceptic', 'deutschmark', 'courting', 'deal', 'withdraw', 'dab', 'withdrawals', 'eurosceptics',
          'surrendering', 'aldous', 'lanarkshire', 'leaving', 'signifying', 'roofs', 'ceded', 'absentia', 'treachery',
          'dollar', 'canada', 'pragmatist', 'oven', 'ready', 'brexiters', 'control', 'capitulation', 'leave', 'referendum',
          'agreement', 'prorogue', 'smoothest', 'depreciate', 'managed', 'mutiny', 'overvalued', 'ideologues', 'foreign',
          'eec', 'war', 'prorogued', 'hannan', 'appease', 'pendolino', 'southbound', 'left', 'line', 'hard', 'bill']
 
no_change = ['prime', 'even', 'parliament', 'care', 'well', 'constituency', 'tax', 'children',
             'business', 'report', 'case', 'sure', 'like', 'see', 'state', 'order', 'back', 'new', 'hope', 'local',
             'secretary', 'public', 'right', 'much', 'say', 'first', 'minister', 'look', 'system', 'whether', 
             'members', 'million', 'good', 'today', 'services', 'clear', 'help', 'time', 'place', 'put', 'last', 'must', 'money', 'one', 
             'way', 'work', 'would', 'think', 'two', 'great', 'could', 'lady', 'us', 'come', 'however', 'may', 'going', 'go',
             'given', 'year', 'might', 'part', 'get', 'make', 'point', 'committee', 'years', 'also', 'know',
             'government', 'take', 'house', 'agree', 'member', 'number', 'across', 'made', 'give', 'gentleman', 'important', 'said',
             'people', 'issue', 'support', 'ensure']


In [12]:
words_of_interest = cosine_similarity_df_sorted[cosine_similarity_df_sorted['Word'].isin(change+no_change)]

words_of_interest.loc[words_of_interest['Word'].isin(change), 'semanticDifference'] = 'change' 
words_of_interest.loc[words_of_interest['Word'].isin(no_change), 'semanticDifference'] = 'no_change' 

In [13]:
change_cossim = words_of_interest.loc[words_of_interest['semanticDifference'] == 'change', 'Cosine_similarity'] 
no_change_cossim = words_of_interest.loc[words_of_interest['semanticDifference'] == 'no_change', 'Cosine_similarity'] 

In [14]:
change_cossim.hist()

In [15]:
no_change_cossim.hist()

In [16]:
words_of_interest['Cosine_similarity'].hist()

In [17]:
words_of_interest['semanticDifference'].value_counts()

# **Logistic Regression**

In [20]:
X = words_of_interest['Cosine_similarity'].values.reshape(-1,1)
y = words_of_interest['semanticDifference']

undersample = RandomUnderSampler(sampling_strategy=1.0)

X_over, y_over = undersample.fit_resample(X, y)
X, y = X_over, y_over

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

logreg = LogisticRegression()
kf = logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print('Y value counts',y.value_counts(),'\n')
print('Y train value counts', y_train.value_counts())

In [21]:
scoring = {'accuracy' : make_scorer(accuracy_score), 
               'precision' : make_scorer(precision_score,pos_label='change'),
               'recall' : make_scorer(recall_score,pos_label='change'), 
               'f1_score' : make_scorer(f1_score,pos_label='change')}

scores = cross_validate(kf, X, y, cv=10, scoring=scoring,error_score='raise')


'''
print('Accuracy', scores['test_accuracy'].mean())
print('Precision', scores['test_precision'].mean())
print('Recall', scores['test_recall'].mean())
print('F1 Score', scores['test_f1_score'].mean())'''


accuracy, precision, recall, f1_score = [], [], [], []

accuracy.append(scores['test_accuracy'].mean())
precision.append(scores['test_precision'].mean())
recall.append(scores['test_recall'].mean())
f1_score.append(scores['test_f1_score'].mean())

In [22]:
scoresDict = {'Model':['Vanilla Model'],'Basis': ['Cosine Similarity'],'Accuracy':accuracy,'Precision':precision,'Recall':recall,'F1Score':f1_score}
scoresDf = pd.DataFrame(scoresDict)
scoresDf

In [23]:
cf = confusion_matrix(y_test, y_pred)
ax = sns.heatmap(cf, annot=True, cmap='Blues')

ax.set_title('Vanilla Model-Cosine Similarity Confusion Matrix \n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['change','no_change'])
ax.yaxis.set_ticklabels(['change','no_change'])

## Display the visualization of the Confusion Matrix.
plt.show()

# **T-Test**

In [24]:
group1= words_of_interest['Cosine_similarity'][words_of_interest['semanticDifference'] == 'change']
group2= words_of_interest['Cosine_similarity'][words_of_interest['semanticDifference'] == 'no_change']

In [25]:
# T Test 

summary, results = rp.ttest(group1= words_of_interest['Cosine_similarity'][words_of_interest['semanticDifference'] == 'change'], group1_name= "change",
                            group2= words_of_interest['Cosine_similarity'][words_of_interest['semanticDifference'] == 'no_change'], group2_name= "no_change")
print(summary)

In [26]:
plt.hist(group1) #change
plt.hist(group2) #no_change
plt.show()

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
from scipy import stats
stats.ttest_ind(group1, group2, equal_var=False)

# **Nearest Neighbours' comparison**

In [27]:
%%time 
neighboursInT1 = []
neighboursInT2 = []

for word in words_of_interest['Word'].to_list():
    
    x = model1.wv.similar_by_word(word,10) 
    y = model2.wv.similar_by_word(word,10)

    x = [tup[0] for tup in x]
    y = [tup[0] for tup in y]
    
    neighboursInT1.append(x)
    neighboursInT2.append(y)
    
words_of_interest['neighboursInT1'] = neighboursInT1
words_of_interest['neighboursInT2'] = neighboursInT2

#words_of_interest['overlappingNeighbours'] = ?
#intersectingNeighbs = set(words_of_interest['neighboursInT1'].to_list()).intersect(words_of_interest['neighboursInT2'].to_list())
lengthOverlap = []

for index in (words_of_interest['neighboursInT1'].index):
    neighboursT1 = words_of_interest.at[index, 'neighboursInT1']
    neighboursT2 = words_of_interest.at[index, 'neighboursInT2']
    lengthOverlap.append(len(set(neighboursT1).intersection(neighboursT2)))

words_of_interest['overlappingNeighbours'] = lengthOverlap

In [28]:
words_of_interest[words_of_interest['semanticDifference']=='change']['overlappingNeighbours'].describe()

In [29]:
words_of_interest[words_of_interest['semanticDifference']=='no_change']['overlappingNeighbours'].describe()

In [30]:
neighbours_of_changed_words = words_of_interest[words_of_interest['semanticDifference']=='change'].sort_values(by='Cosine_similarity',ascending=True)[['Word','neighboursInT1','neighboursInT2']]

In [45]:
#out_path = './'
#neighbours_of_changed_words.to_csv(os.path.join(out_path, 'neighbours_of_changed_words.csv'), encoding='utf-8', columns = ['Word', 'neighboursInT1', 'neighboursInT2'])

In [31]:
X = words_of_interest['overlappingNeighbours'].values.reshape(-1,1)
y = words_of_interest['semanticDifference']

undersample = RandomUnderSampler(sampling_strategy=1.0)

X_over, y_over = undersample.fit_resample(X, y)
X=X_over
y=y_over

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

logreg = LogisticRegression()
kf = logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print('Y value counts',y.value_counts(),'\n')
print('Y train value counts', y_train.value_counts())

In [32]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

In [33]:
scoring = {'accuracy' : make_scorer(accuracy_score), 
               'precision' : make_scorer(precision_score,pos_label='change'),
               'recall' : make_scorer(recall_score,pos_label='change'), 
               'f1_score' : make_scorer(f1_score,pos_label='change')}

scores = cross_validate(kf, X, y, cv=10, scoring=scoring,error_score='raise')
accuracy, precision, recall, f1_score = [], [], [], []

print('Accuracy', scores['test_accuracy'].mean())
print('Precision', scores['test_precision'].mean())
print('Recall', scores['test_recall'].mean())
print('F1 Score', scores['test_f1_score'].mean())

accuracy.append(scores['test_accuracy'].mean())
precision.append(scores['test_precision'].mean())
recall.append(scores['test_recall'].mean())
f1_score.append(scores['test_f1_score'].mean())



In [34]:
scoresDict = {'Model':['Vanilla Model'], 'Basis': ['Overlap of nearest 10 neighbours'], 'Accuracy':accuracy, 'Precision':precision, 'Recall': recall, 'F1Score': f1_score}
scoresDf = pd.concat([scoresDf, pd.DataFrame(scoresDict)])
scoresDf

In [53]:
#print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
# .72 for 3 neighbours
# .68 accuracy for 5 neighbours
# .62 for 10 neighbours
# .55 for 100 neighbours
# .47 for 500 neighbours

In [36]:
cf_neighb_10 = confusion_matrix(y_test, y_pred)
ax = sns.heatmap(cf_neighb_10, annot=True, cmap='Blues')

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['change','no_change'])
ax.yaxis.set_ticklabels(['change','no_change'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [37]:
group1= words_of_interest['overlappingNeighbours'][words_of_interest['semanticDifference'] == 'change']
group2= words_of_interest['overlappingNeighbours'][words_of_interest['semanticDifference'] == 'no_change']


In [38]:
# T Test with 10 neighbours

summary_neighbours, results_neighbours = rp.ttest(group1= words_of_interest['overlappingNeighbours'][words_of_interest['semanticDifference'] == 'change'], group1_name= "change",
                            group2= words_of_interest['overlappingNeighbours'][words_of_interest['semanticDifference'] == 'no_change'], group2_name= "no_change")
print(summary_neighbours)

In [39]:
plt.hist(group1) #change
plt.hist(group2) #no_change
plt.show()

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
from scipy import stats
stats.ttest_ind(group1, group2, equal_var=False)